In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import data_request as dr

In [14]:
df = dr.retrieve_data_request()
df

,output variable name,units,ag,long_name,standard_name,priority,comments,frequency
0,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,mon
1,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,day
2,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,1hr
3,tasmax,K,NaN,Daily Maximum Near-Surface Air Temperature,air_temperature,CORE,maximum from all integrated time steps per day,mon
4,tasmax,K,NaN,Daily Maximum Near-Surface Air Temperature,air_temperature,CORE,maximum from all integrated time steps per day,day
...,...,...,...,...,...,...,...,...
582,rootd,m,NaN,Maximum Root Depth,root_depth,TIER 2,NaN,fx
583,sftlaf,%,NaN,Percentage of the Grid Cell Occupied by Lake,lake_area_fraction,TIER 2,not in CMIP or in CF,fx
584,sfturf,%,NaN,Percentage of the Grid Cell Occupied by City,urban_area_fraction,TIER 2,not in CMIP or in CF,fx
585,dtb,m,NaN,Depth to Bedrock,bedrock_depth,TIER 2,not in CMIP or in CF (lower boundary of land s...,fx


In [4]:
df.groupby(["priority", "output variable name"])["frequency"].apply(list)

priority  output variable name
CORE      clt                     [mon, day, 1hr]
          evspsbl                 [mon, day, 1hr]
          hurs                    [mon, day, 1hr]
          huss                    [mon, day, 1hr]
          orog                               [fx]
                                       ...       
TIER 2    zg20                    [mon, day, 6hr]
          zg30                    [mon, day, 6hr]
          zg50                    [mon, day, 6hr]
          zg70                    [mon, day, 6hr]
          zg750                   [mon, day, 6hr]
Name: frequency, Length: 199, dtype: object

In [15]:
df.to_csv("cordex-cmip6-data-request.csv", index=False)